In [3]:
# Julia test

# Pkg.add("XMLDict")

In [9]:
file_path = "/home/fa2k/nsc/statfiles/170111_D00132_0218_ACAHFEANXX/Data/Intensities/BaseCalls/Stats/DemultiplexingStats.xml"
my_data = readall(file_path)
len = length(my_data)
print("F: $len")

F: 28326

In [37]:
Pkg.add("XMLDict")

INFO: Nothing to be done


In [38]:
using XMLDict

In [39]:
@time data = parse_xml(my_data)

MethodError: MethodError: `length` has no method matching length(::XMLDict.XMLDictElement)

0.000627 seconds (6 allocations: 384 bytes)


## Number of reads and index per sample

Data structure: Coordinates = (lane, index)


In [40]:
num_reads = Dict()

for project = data["Flowcell"]["Project"]
    for sample = project["Sample"]
        for barcode = sample["Barcode"]
            barcode_name = barcode[:name]
            for lane = barcode["Lane"]
                lane_num = parse(Int, lane[:number])
                print(barcode_name)
                count = get(num_reads, (lane_num, barcode_name), 0)
                num_reads[(lane_num, barcode_name)] = count + 1
            end
        end
    end
end

unknownunknown

LoadError: LoadError: MethodError: `start` has no method matching start(::XMLDict.XMLDictElement)
while loading In[40], in expression starting on line 3

unknownunknownunknownunknownunknownunknownallallallallallallallall

In [28]:
num_reads

Dict{Any,Any} with 16 entries:
  (4,"all")     => 1
  (8,"unknown") => 1
  (1,"unknown") => 1
  (3,"all")     => 1
  (2,"all")     => 1
  (6,"unknown") => 1
  (7,"unknown") => 1
  (2,"unknown") => 1
  (8,"all")     => 1
  (4,"unknown") => 1
  (3,"unknown") => 1
  (1,"all")     => 1
  (5,"all")     => 1
  (5,"unknown") => 1
  (6,"all")     => 1
  (7,"all")     => 1

## LibExpat version

In [35]:
Pkg.update()

INFO: Updating METADATA...
INFO: Computing changes...
INFO: No packages to install, update or remove


In [36]:
Pkg.add("LibExpat")

INFO: Nothing to be done


In [30]:
using LibExpat

In [33]:
@time xmltree = xp_parse(my_data);

  0.003265 seconds (17.04 k allocations: 1.134 MB)


In [109]:
mismatch_xpath = "Flowcell/Project[@name != \"all\"]/Sample/Barcode[@name != \"all\"]/Lane/BarcodeCount"
mismatch_results = xmltree[mismatch_xpath]
perfect_xpath = "Flowcell/Project[@name != \"all\"]/Sample/Barcode[@name != \"all\"]/Lane/PerfectBarcodeCount"
perfect_results = xmltree[perfect_xpath]

40-element Array{LibExpat.ETree,1}:
 <PerfectBarcodeCount>8437774</PerfectBarcodeCount> 
 <PerfectBarcodeCount>7623996</PerfectBarcodeCount> 
 <PerfectBarcodeCount>7518344</PerfectBarcodeCount> 
 <PerfectBarcodeCount>7746829</PerfectBarcodeCount> 
 <PerfectBarcodeCount>8306579</PerfectBarcodeCount> 
 <PerfectBarcodeCount>7302641</PerfectBarcodeCount> 
 <PerfectBarcodeCount>7132213</PerfectBarcodeCount> 
 <PerfectBarcodeCount>6958048</PerfectBarcodeCount> 
 <PerfectBarcodeCount>36748469</PerfectBarcodeCount>
 <PerfectBarcodeCount>37728080</PerfectBarcodeCount>
 <PerfectBarcodeCount>39215881</PerfectBarcodeCount>
 <PerfectBarcodeCount>39202890</PerfectBarcodeCount>
 <PerfectBarcodeCount>36279342</PerfectBarcodeCount>
 ⋮                                                  
 <PerfectBarcodeCount>36931856</PerfectBarcodeCount>
 <PerfectBarcodeCount>37323427</PerfectBarcodeCount>
 <PerfectBarcodeCount>40182541</PerfectBarcodeCount>
 <PerfectBarcodeCount>30420986</PerfectBarcodeCount>
 <PerfectB

In [110]:
read_counts = Dict()
perfect_read_counts = Dict()

for res = mismatch_results
    lane = parse(Int,res.parent.attr["number"])
    barcode = res.parent.parent.attr["name"]
    
    read_counts[(lane, barcode)] = parse(Int64, res.elements[1])
end

In [111]:
read_counts

Dict{Any,Any} with 40 entries:
  (2,"CCGAAGTA") => 37356133
  (1,"AGGCTAAC") => 39992995
  (2,"CGCATACA") => 31647030
  (6,"AATCCGTC") => 31093897
  (8,"ACACAGAA") => 41194726
  (3,"AAGAGATC") => 41398836
  (8,"unknown")  => 6958048
  (1,"unknown")  => 8437774
  (1,"CTCAATGA") => 37369606
  (2,"AGATGTAC") => 37017974
  (3,"AATGTTGC") => 44894096
  (3,"AGATCGCA") => 35033696
  (8,"GAACAGGC") => 36034893
  (8,"AACCGAGA") => 41106212
  (4,"TGGAACAA") => 36750785
  (6,"unknown")  => 7302641
  (7,"AAGACGGA") => 36506769
  (7,"unknown")  => 7132213
  (7,"CACCTTAC") => 37752922
  (4,"CAACCACA") => 39787285
  (2,"unknown")  => 7623996
  (4,"ACAGATTC") => 40467000
  (6,"AACAACCA") => 40785672
  (7,"TTCACGCA") => 35412455
  (8,"ACAAGCTA") => 41380292
  ⋮              => ⋮

In [101]:
?LibExpat.ETree

No documentation found.

**Summary:**

```julia
type LibExpat.ETree <: Any
```

**Fields:**

```julia
name     :: AbstractString
attr     :: Dict{AbstractString,AbstractString}
elements :: Array{Union{AbstractString,LibExpat.ETree},1}
parent   :: LibExpat.ETree
```
